<a href="https://colab.research.google.com/github/Aksinhaa/Pony/blob/main/NGS_collab_snp_identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Variant calling is the process of identifying genetic variants, such as single nucleotide polymorphisms (SNPs) and small insertions or deletions (indels) from next-generation sequencing (NGS) data. It involves comparing sequencing reads from an individual to a reference genome to detect differences, like single nucleotide variants (SNVs). In this tutorial, we will focus specifically on detecting SNPs and indels.

In this step:

a) Call variants (Single Nucleotide Polymorphisms and Indels) using tools like bcftools or strelka

b) Produce a VCF file (Variant Call Format) with detailed information about each variant

c) Optionally apply variant filtering to remove false positives or low-confidence variants

Why it's important: This is the core step of population genetics—identifying the genetic differences across sample.


For this tutorial we are going to use Strelka, a tool utilised for germline and somatic variant calling.

1: Germline Calling: Utilizes haplotype-based model to accurately detect inherited variants.

(Haplotype is a set of DNA variants inherited together on the same chromosome copy)

2: Somatic Calling: identifying genetic mutations that arise in somatic (non-germline) cells. These mutations are not inherited from parents and do not get passed on to offspring.

Workflow Execution: Strelka2 can be run in two steps: configuration (specifying input data) and execution (specifying parameters).


The first step is to download the Miniconda installer for Linux using `wget`.


In [ ]:
%%bash
# Download and install Miniconda
wget -q https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O miniconda.sh
bash miniconda.sh -b -p /usr/local/miniconda

# Add conda to PATH and initialize shell
export PATH="/usr/local/miniconda/bin:$PATH"
source /usr/local/miniconda/etc/profile.d/conda.sh

# Accept Terms of Service for required conda channels
conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main
conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r

# Create environment and install Strelka + Samtools
conda create -y -n strelka -c bioconda strelka samtools

In [ ]:
%%bash
export PATH="/usr/local/miniconda/bin:$PATH"
source /usr/local/miniconda/etc/profile.d/conda.sh

conda list -n strelka



For the variant calling step, we will download the BAM file which is aligned, sorted, and duplicate-marked file (generated during the mapping step), from the Zenodo repository. This will ensure that they are ready for use in downstream variant identification.


In [ ]:
%%bash
export PATH="/usr/local/miniconda/bin:$PATH"
source /usr/local/miniconda/etc/profile.d/conda.sh
conda activate strelka

which configureStrelkaGermlineWorkflow.py

In [ ]:
%%bash
rm -rf reference

# Create the directory if it doesn't exist
mkdir -p reference

# URL for reference genome tar.gz file
wget -P reference  https://zenodo.org/records/14258052/files/GCA_021130815.1_PanTigT.MC.v3_genomic.fna

# List extracted contents
ls -F reference/

Once we have our required BAM files, we will start the step for variant calling. In this analysis we use the Strelka germline workflow, which is initiated through the configureStrelkaGermlineWorkflow.py script.

This germline configuration file contains:

1: Information about the input BAM files (aligned, sorted, and duplicate-marked reads).

2: The reference genome path, which Strelka uses for aligning sequences and identifying variant sites.

3: Workflow parameters, such as filtering settings, runtime options, and rules for how Strelka processes sequencing data.

4: Module definitions, specifying the order and structure of the processing steps used to detect SNVs and small indels.

5: Paths to output directories, where Strelka will write variant call results, logs, and intermediate files.

In short, this germline configuration is used by Strelka to build and execute the variant calling workflow. It ensures that the pipeline runs consistently with the correct inputs, reference genome, and settings needed for accurate germline variant detection.

In [ ]:
%%bash
# Create bam_files directory if it doesn't exist
mkdir -p bam_files

# Move the .fna file from reference/ to bam_files/
mv reference/GCA_021130815.1_PanTigT.MC.v3_genomic.fna bam_files/

# List contents of bam_files to confirm
ls -F bam_files

In [ ]:
%%bash
# Activate conda environment that contains samtools
export PATH="/usr/local/miniconda/bin:$PATH"
source /usr/local/miniconda/etc/profile.d/conda.sh
conda activate strelka   # or the environment where samtools is installed

# Move into bam_files directory
cd bam_files

# Index the reference FASTA
samtools faidx GCA_021130815.1_PanTigT.MC.v3_genomic.fna

echo "Indexed reference file: $fna_file"
ls -F


In [ ]:
%%bash
# Create directory for BAM files
mkdir -p bam_files

# Download all BAM and BAI files into bam_files/
wget -q "https://zenodo.org/records/17895817/files/BEN_CI16_aligned_reads_deduplicated.bam?download=1"     -O bam_files/BEN_CI16_aligned_reads_sorted_deduplicated.bam
wget -q "https://zenodo.org/records/17895817/files/BEN_CI16_aligned_reads_deduplicated.bam.bai?download=1" -O bam_files/BEN_CI16_aligned_reads_sorted_deduplicated.bam.bai

wget -q "https://zenodo.org/records/17895817/files/BEN_NW10_aligned_reads_deduplicated.bam?download=1"     -O bam_files/BEN_NW10_aligned_reads_sorted_deduplicated.bam
wget -q "https://zenodo.org/records/17895817/files/BEN_NW10_aligned_reads_deduplicated.bam.bai?download=1" -O bam_files/BEN_NW10_aligned_reads_sorted_deduplicated.bam.bai

wget -q "https://zenodo.org/records/17895817/files/BEN_SI18_aligned_reads_deduplicated.bam?download=1"     -O bam_files/BEN_SI18_aligned_reads_sorted_deduplicated.bam
wget -q "https://zenodo.org/records/17895817/files/BEN_SI18_aligned_reads_deduplicated.bam.bai?download=1" -O bam_files/BEN_SI18_aligned_reads_sorted_deduplicated.bam.bai

# List downloaded files
ls -lh bam_files





After running the Strelka germline variant calling workflow, the pipeline produces a set of output files, most importantly the **VCF (Variant Call Format) files**. These files contain the list of single nucleotide variants (SNVs) and small insertions/deletions (indels) identified from the input BAM files.

Once the VCF files are generated, they represent the final variant calls produced by Strelka, and they can now be used for downstream analyses. Typical post-processing steps may include:

* **Quality assessment** of the variants by inspecting Strelka’s FILTER and quality score annotations.
* **Merging or comparing VCFs** (if multiple samples were processed) to evaluate shared or unique variants.
* **Functional annotation** using external tools (e.g., VEP, SnpEff) to predict gene impacts, consequences, or biological relevance.
* **Filtering based on depth, genotype quality, or allele frequency**, depending on the research goals.

In summary, the generation of VCF files marks the completion of the variant calling workflow, providing a structured dataset of all detected variants. These files form the foundation for interpretation, annotation, and further biological or population-level analyses.


In [ ]:
%%bash
# Activate environment
export PATH="/usr/local/miniconda/bin:$PATH"
source /usr/local/miniconda/etc/profile.d/conda.sh
conda activate strelka

# Create the run directory manually (recommended for clarity)
mkdir -p strelka_run

# Configure Strelka workflow
configureStrelkaGermlineWorkflow.py \
    --bam bam_files/BEN_CI16_aligned_reads_sorted_deduplicated.bam \
    --bam bam_files/BEN_NW10_aligned_reads_sorted_deduplicated.bam \
    --bam bam_files/BEN_SI18_aligned_reads_sorted_deduplicated.bam \
    --referenceFasta bam_files/GCA_021130815.1_PanTigT.MC.v3_genomic.fna \
    --runDir strelka_run

In [ ]:
%%bash
# Activate environment
export PATH="/usr/local/miniconda/bin:$PATH"
source /usr/local/miniconda/etc/profile.d/conda.sh
conda activate strelka

strelka_run/runWorkflow.py -m local -j 4